First I need to get my data 

In [980]:
import os
import pandas as pd
import numpy as np
import mypytable as mypy
from sklearn.model_selection import train_test_split
import tensorflow as tf
import copy


In [981]:
stockfiles= os.listdir("/home/cbarker4/Documents/DataScience/StockTrader/Data")
# print(stockfiles)

In [982]:
X = []
Y=[]
Xlast=[]
stockfiles=["AAPL.csv"]
for j,val in enumerate(stockfiles):
    print(val)
    if j == 30:
        break

    mt = mypy.MyPyTable()
    mt.load_from_file("/home/cbarker4/Documents/DataScience/StockTrader/Data/"+val)
    # mt.drop_column('v')
    # mt.drop_column('t')
    # mt.drop_column('s')
    # mt.drop_column('h')
    # mt.drop_column('l')
    # mt.drop_column('o')

    mt.drop_column(0)
    i = 100
    while i <  len(mt.data)-1:
    

        table = mt.create_sub_table(i-100,i)
        # big = table.max()
        # little = table.min()
        # table = table.normalize()
        

      
        # print(table.data[0])
        X.append(copy.deepcopy(table.get_column(0)))
        # Xlast.append(table.data[-1])
        # num = int((((mt.get_row(i+1)[0])-little))/(big-little)*1000)/10
        num = mt.get_row(i+1)[0]
        Y.append(num)
        i+=1
    


AAPL.csv


In [983]:
X = np.array(X)
# print(Y)
Y = np.array(Y)
# print(Y.shape)
count = 0
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.25 ,random_state=10)
i=0
for val in X_train:
    # print(val[-1][0])
    # if val[-1][0]<Y_train[i]:
    if val[-1]<Y_train[i]:
        count+=1
    i+=1

print(X_train[-1][-1])
print(Y_train[-1])
print(count/len(Y_train))



152.37
154.48
0.4864864864864865


In [984]:
# Check shapes
print('X_train shape: ', X_train.shape)
print('y_train shape: ', Y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', Y_test.shape)

X_train shape:  (111, 100)
y_train shape:  (111,)
X_test shape:  (38, 100)
y_test shape:  (38,)


In [985]:
def accuracy(y_true,y_pred):

    good = abs(y_true-y_pred)
    if good>.25:
        False
    else:
        True
    


# Build epoch checkpoint callback
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=500,
        verbose=1,
        min_delta = 0,
        mode='auto',
        baseline=None,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join("/home/cbarker4/Documents/DataScience/StockTrader/Model", 'ckpt', "{epoch:02d}-{loss:.2f}.hdf5"),
        monitor='loss',
        verbose = 0,
        save_best_only = False,
        save_weights_only = False,
        mode = 'auto',
        save_freq='epoch',
        options=None,
        initial_value_threshold=None
    )   
]

In [986]:
class Custom_Ce_Loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call (self, y_true,y_pred):
        # if y_true[1]:
        #     if y_pred>y_true[0]:
        ans = tf.math.subtract(y_true[0],y_pred)
        ans = tf.math.square(ans)
        return tf.math.abs(ans)
        #     else:
        #         ans = tf.math.subtract(y_true[0],y_pred)
        #         ans =  tf.math.abs(ans)
        #         return tf.math.square(ans)
        # else:
        #     if y_pred<y_true[0]:
        #         ans = tf.math.subtract(y_true[0],y_pred)
        #         return tf.math.abs(ans)
        #     else:
        #         ans = tf.math.subtract(y_true[0],y_pred)
        #         ans =  tf.math.abs(ans)
        #         return tf.math.square(ans)

        

In [987]:
# loss_fun=["mean_absolute_error","KLD","MAE","MAPE","MSE","MSLE","binary_crossentropy","binary_focal_crossentropy","binary_crossentropy","binary_focal_crossentropy","categorical_accuracy","categorical crossentropy","deserialize","get","logcosh","hinge","kld","poisson","mse","mean_squared_logarithmic_error"]
# loss_fun = ["binary_crossentropy","binary_focal_crossentropy"]
# loss_fun = ["binary_crossentropy","binary_focal_crossentropy","categorical_accuracy","categorical crossentropy","deserialize"]
# loss_fun = ["mean_squared_logarithmic_error"]
loss_fun = ["MAE"]
best = {}
for losfun in loss_fun:
    # break

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(units=64,
                                return_sequences=True,
                                input_shape=(X_train.shape[1],1)))
    # model.add(tf.keras.layers.LSTM(units=256,return_sequences=True))
    model.add(tf.keras.layers.LSTM(64))
    model.add(tf.keras.layers.Dense(32))
    # model.add(tf.keras.layers.Dense(64))
    # model.add(tf.keras.layers.Dense(32))

    model.add(tf.keras.layers.Dropout(0.5))
    
    model.add(tf.keras.layers.Dense(1))
    model.summary()


    

    # print(Y_train)
    
    

Model: "sequential_106"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_230 (LSTM)             (None, 100, 64)           16896     
                                                                 
 lstm_231 (LSTM)             (None, 64)                33024     
                                                                 
 dense_284 (Dense)           (None, 32)                2080      
                                                                 
 dropout_94 (Dropout)        (None, 32)                0         
                                                                 
 dense_285 (Dense)           (None, 1)                 33        
                                                                 
Total params: 52,033
Trainable params: 52,033
Non-trainable params: 0
_________________________________________________________________


In [988]:
model.compile(optimizer='adam',
              loss='mean_squared_error')
model.fit(X_train,Y_train,epochs=1000, callbacks=callbacks)


Epoch 1/1000
4/4 [==============================] - 2s 13ms/step - loss: 21461.7656
Epoch 2/1000
4/4 [==============================] - 0s 15ms/step - loss: 20996.7676
Epoch 3/1000
4/4 [==============================] - 0s 16ms/step - loss: 20400.5898
Epoch 4/1000
4/4 [==============================] - 0s 13ms/step - loss: 19690.0723
Epoch 5/1000
4/4 [==============================] - 0s 13ms/step - loss: 18889.2988
Epoch 6/1000
4/4 [==============================] - 0s 11ms/step - loss: 18213.1230
Epoch 7/1000
4/4 [==============================] - 0s 12ms/step - loss: 17323.9805
Epoch 8/1000
4/4 [==============================] - 0s 15ms/step - loss: 16490.2734
Epoch 9/1000
4/4 [==============================] - 0s 13ms/step - loss: 15627.5781
Epoch 10/1000
4/4 [==============================] - 0s 16ms/step - loss: 14738.9795
Epoch 11/1000
4/4 [==============================] - 0s 13ms/step - loss: 13897.0947
Epoch 12/1000
4/4 [==============================] - 0s 115ms/step - loss:

In [ ]:
y_pred = model.predict(X_test)
print(y_pred)

2/2 [==============================] - 0s 3ms/step
[[124.78445 ]
 [142.37645 ]
 [141.13268 ]
 [126.242485]
 [140.52933 ]
 [139.5753  ]
 [141.45914 ]
 [126.79354 ]
 [137.10976 ]
 [148.43422 ]
 [148.13646 ]
 [145.60222 ]
 [135.76834 ]
 [144.30215 ]
 [135.71251 ]
 [143.32448 ]
 [126.23312 ]
 [140.63069 ]
 [134.74687 ]
 [138.59497 ]
 [136.20602 ]
 [139.63004 ]
 [139.96138 ]
 [137.63286 ]
 [134.12299 ]
 [143.46051 ]
 [124.407875]
 [146.36311 ]
 [135.64096 ]
 [124.543335]
 [142.10228 ]
 [137.4646  ]
 [136.77147 ]
 [141.20082 ]
 [136.688   ]
 [133.2855  ]
 [134.21478 ]
 [137.2618  ]]


In [ ]:

Xlast=[]
correct = 0
for val in X_test:
    Xlast.append(val[-1])


correct = 0
removed = 0
falsepos = 0
pos = 0
neg=0
falseneg =0
for i,val in enumerate(Y_test):
    if i < 10:
        print(Xlast[i],Y_test[i],y_pred[i])
    if Xlast[i]!=y_pred[i]:
        if Xlast[i] > val:

            if Xlast[i]>y_pred[i]:
                pos+=1
                correct+=1
            else :
                neg+=1
                falseneg+=1
        
        else:
            if Xlast[i]<y_pred[i]:
                correct+=1 
                neg+=1
            else:
                pos+=1
                falsepos+=1
    else:
        removed+=1

print('Correct predictions: ', correct)
print('Incorrect predictions: ', (len(y_pred)-removed) - correct)
print('Accuracy: ', correct / (len(y_pred)-removed))
print('False Pos:',falsepos)
print('Pos:',pos)
print('Pos Accuracy',(pos-falsepos)/pos)
print('Neg:',neg)
print('Neg Accuracy',(neg-falseneg)/neg)

126.04 129.93 [124.78445]
152.59 155.85 [142.37645]
150.7 156.9 [141.13268]
129.62 130.73 [126.242485]
150.04 150.72 [140.52933]
148.01 151.07 [139.5753]
151.07 144.22 [141.45914]
129.93 126.36 [126.79354]
144.8 153.34 [137.10976]
165.63 164.66 [148.43422]
Correct predictions:  16
Incorrect predictions:  22
No change Correct: 0
Accuracy:  0.42105263157894735
Accuracy:  0.42105263157894735
False Pos: 22
Pos: 38
Pos Accuracy 0.42105263157894735
0


ZeroDivisionError: division by zero